# Export flux variables during 16-19 July 2022
- This script is used to export daytime and nighttime flux variables during 16-19 July 2022.
- Simulations: GM_SLUCM, GM_CLMU. 

In [35]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [36]:
ds_surf = xr.open_dataset('/gws/nopw/j04/duicv/yuansun/0_WRFvsWRF-CTSM/input/WRF-CTSM/surfdata_1.2x1.2_SSP5-8.5_2022_78pfts_c250607.nc')
ds_urban = ds_surf['PCT_URBAN'].sum(dim='numurbl')
flat_index = ds_urban.argmax().item()

# Convert flat index back to (lsmlat, lsmlon) index
max_lat_idx, max_lon_idx = np.unravel_index(flat_index, ds_urban.shape)

# Get coordinate values
max_lat = ds_urban['lsmlat'].values[max_lat_idx]
max_lon = ds_urban['lsmlon'].values[max_lon_idx]
print(f"Max value: {ds_urban.max().item()} at lat={max_lat}, lon={max_lon}")

Max value: 88.24227244932186 at lat=18, lon=30


In [40]:
i1, i2 = 23, 36 # lon
j1, j2 = 12, 25 # lat
center_i, center_j = 30, 18

In [43]:
ds_urban[j1:j2, i1:i2].mean().values

array(70.86024394)

In [48]:
ds_wrfinput = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/input/WRF-CTSM/d04/wrfinput_d01')
ds_wrfinput_urban = ds_wrfinput['FRC_URB2D']
ds_wrfinput_urban[0, j1:j2, i1:i2].mean().values

array(0.74556214, dtype=float32)

In [ ]:
var_list = ['SWDOWN', 'GLW', 'HFX', 'LH', 'ALBEDO', 'EMISS', 'GRDFLX', 'TSK', 'NOAHRES', 'T2', 'U10', 'V10', 'Q2', 'PSFC']
#var_list = ['QFX','T2', 'Q2', 'HGT', 'PBLH']
date_list = ['07-16', '07-17', '07-18', '07-19']
day_time = '13'
night_time = '01'

model_list = ['wrf', 'wrf-ctsm']
location_list = ['sp', 'region']
time_list = []

for model in model_list: 
    for date in ['07-16', '07-17', '07-18', '07-19']:
        if model =='wrf':
            if date in ['07-16', '07-17']:
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_wrf/wrfout_d01_2022-07-13_01:00:00.nc')
            else:
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_wrf/wrfout_d01_2022-07-18_01:00:00.nc')    
        elif model == 'wrf-ctsm':
            ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_wrf-ctsm/wrfout_d01_2022-07-15_01:00:00.nc')
        day_time = pd.to_datetime(f'2022-{date} 13:00:00')
        night_time = pd.to_datetime(f'2022-{date} 01:00:00')
        time = pd.to_datetime(ds_wrf['XTIME'].values)
        idx_day = np.where(time == day_time)[0].item()
        print(idx_day, time[idx_day])
        idx_night = np.where(time == night_time)[0].item()
        day_sp = []
        day_region = []
        night_sp = []
        night_region = []
        for var in var_list:
            mean_day_region = ds_wrf[var][idx_day:idx_day+2, j1:j2, i1:i2].mean().item() # time, lat, lon
            mean_day_sp = ds_wrf[var][idx_day:idx_day+2, center_j, center_i].mean().item()
            day_sp.append(mean_day_sp)
            day_region.append(mean_day_region)
            mean_night_region = ds_wrf[var][idx_night:idx_night+2, j1:j2, i1:i2].mean().item() # time, lat, lon
            mean_night_sp = ds_wrf[var][idx_night:idx_night+2, center_j, center_i].mean().item()
            night_sp.append(mean_night_sp)
            night_region.append(mean_night_region)
        df_rows = pd.DataFrame([day_sp, day_region, night_sp, night_region], columns=var_list)
        df_rows['time'] = [day_time, day_time, night_time, night_time]
        df_rows['model'] = model
        df_rows['location'] = ['sp', 'region', 'sp', 'region']
        df_rows['tag'] = ['13:30', '13:30', '01:30', '01:30']
        time_list.append(df_rows)
df_time = pd.concat(time_list, ignore_index=True) 
df_time.head()

84 2022-07-16 13:00:00
108 2022-07-17 13:00:00
12 2022-07-18 13:00:00
36 2022-07-19 13:00:00
36 2022-07-16 13:00:00
60 2022-07-17 13:00:00
84 2022-07-18 13:00:00
108 2022-07-19 13:00:00


,SWDOWN,GLW,HFX,LH,ALBEDO,EMISS,GRDFLX,TSK,NOAHRES,T2,U10,V10,Q2,PSFC,time,model,location,tag
0,836.667725,328.175537,319.074921,3.122260,0.154772,0.980000,-154.159576,307.841919,-8.616257,295.416748,3.342315,-0.492369,0.006128,102002.820312,2022-07-16 13:00:00,wrf,sp,13:30
1,833.092529,327.832550,301.654022,33.093815,0.166591,0.971141,-142.642197,307.409180,-6.365817,295.259613,3.105556,-0.778966,0.006242,101860.500000,2022-07-16 13:00:00,wrf,region,13:30
2,0.000000,295.979919,5.677783,-0.171625,0.018000,0.980000,88.613800,286.359009,-0.373672,285.903564,1.688490,0.017660,0.006841,102009.140625,2022-07-16 01:00:00,wrf,sp,01:30
3,0.000000,294.687195,5.096761,-0.236937,0.053288,0.971141,83.788857,285.335175,-0.416177,285.237152,1.737798,-0.266600,0.006764,101867.929688,2022-07-16 01:00:00,wrf,region,01:30
4,552.928711,384.764587,205.314087,2.190381,0.154772,0.980000,-108.935753,307.570679,-3.382828,300.324951,1.414035,4.726068,0.006736,101744.515625,2022-07-17 13:00:00,wrf,sp,13:30


In [38]:
def compute_rh2_from_wrf(Q2, T2 , PSFC):
    # Convert T2 from Kelvin to Celsius
    T2_C = T2 - 273.15
    # Saturation vapor pressure (hPa)
    es = 6.112 * np.exp((17.67 * T2_C) / (T2_C + 243.5))

    # Actual vapor pressure (hPa), using PSFC (Pa)
    e = (Q2 * PSFC) / (0.622 + Q2) / 100  # Convert from Pa to hPa

    # Relative Humidity (%)
    RH2 = (e / es) * 100
    RH2 = np.clip(RH2, 0, 100)  # Ensure values are within 0-100%
    return RH2

In [39]:
df_time_average = df_time.groupby(['tag', 'model', 'location']).mean().reset_index().copy() 
df_time_average['SWUP'] = df_time_average['SWDOWN'] * df_time_average['ALBEDO']
# Stefan-Boltzmann constant (W/m²/K⁴)
stefan_boltzmann = 5.67e-8
df_time_average['LWUP'] = df_time_average['EMISS'] * stefan_boltzmann * np.power(df_time_average['TSK'],4) # ref: https://forum.mmm.ucar.edu/threads/wrf-energy-and-moisture-budget.5674/#post-26129
df_time_average['LWDOWN'] = df_time_average['EMISS'] * df_time_average['GLW']
#df_time_average['netLW'] = df_time_average['EMISS'] * (df_time_average['GLW'] - df_time_average['LWUP'])
df_time_average['netLW'] = df_time_average['LWDOWN'] - df_time_average['LWUP']
df_time_average['NET'] = df_time_average['SWDOWN'] - df_time_average['SWUP'] + df_time_average['netLW']
df_time_average['GRD'] = df_time_average['NET'] - df_time_average['HFX'] - df_time_average['LH'] 
df_time_average['RH2M'] = compute_rh2_from_wrf(df_time_average['Q2'], df_time_average['T2'], df_time_average['PSFC'])
df_time_average['W10M'] = np.sqrt(df_time_average['U10']**2 + df_time_average['V10']**2)
df_time_average

,tag,model,location,SWDOWN,GLW,HFX,LH,ALBEDO,EMISS,GRDFLX,...,PSFC,time,SWUP,LWUP,LWDOWN,netLW,NET,GRD,RH2M,W10M
0,01:30,wrf,region,0.000000,348.866905,2.044273,0.040389,0.053288,0.971141,69.158785,...,101484.601562,2022-07-17 13:00:00,0.000000,405.344946,338.798969,-66.545977,-66.545977,-68.630639,49.618172,0.815662
1,01:30,wrf,sp,0.000000,350.552872,2.946615,-0.038948,0.018000,0.980000,74.658381,...,101627.517578,2022-07-17 13:00:00,0.000000,415.419271,343.541821,-71.877450,-71.877450,-74.785118,47.709363,0.645768
2,01:30,wrf-ctsm,region,0.000000,345.692520,-6.751510,1.175791,0.300000,1.000000,0.000000,...,101489.017578,2022-07-17 13:00:00,0.000000,404.872779,345.692520,-59.180258,-59.180258,-53.604539,54.397218,0.699990
3,01:30,wrf-ctsm,sp,0.000000,347.329132,-5.905338,0.274503,0.300000,1.000000,0.000000,...,101632.197266,2022-07-17 13:00:00,0.000000,409.267839,347.329132,-61.938707,-61.938707,-56.307873,52.112415,0.787207
4,13:30,wrf,region,761.343674,369.882721,261.777489,34.837578,0.166591,0.971141,-133.966965,...,101181.816406,2022-07-18 01:00:00,126.832975,524.906627,359.208290,-165.698337,468.812361,172.197294,25.464890,3.750203
5,13:30,wrf,sp,759.691040,370.210983,279.180534,2.531059,0.154772,0.980000,-143.020090,...,101321.359375,2022-07-18 01:00:00,117.579169,533.919043,362.806771,-171.112272,470.999598,189.288005,24.866259,3.768751
6,13:30,wrf-ctsm,region,757.197693,370.730644,274.873180,90.316739,0.141429,1.000000,0.000000,...,101175.244141,2022-07-18 01:00:00,107.089537,539.696499,370.730644,-168.965855,481.142301,115.952381,23.080747,3.288378
7,13:30,wrf-ctsm,sp,763.619705,371.351341,276.730949,93.419388,0.132765,1.000000,0.000000,...,101314.542969,2022-07-18 01:00:00,101.382140,544.611765,371.351341,-173.260423,488.977142,118.826805,22.666465,3.161051


In [49]:
df_time_average.to_csv('./data_for_figure/fluxs.csv', index=False)
df_time_average.head()

,tag,model,location,SWDOWN,GLW,HFX,LH,ALBEDO,EMISS,GRDFLX,...,PSFC,time,SWUP,LWUP,LWDOWN,netLW,NET,GRD,RH2M,W10M
0,01:30,wrf,region,0.000000,348.866905,2.044273,0.040389,0.053288,0.971141,69.158785,...,101484.601562,2022-07-17 13:00:00,0.000000,405.344946,338.798969,-66.545977,-66.545977,-68.630639,49.618172,0.815662
1,01:30,wrf,sp,0.000000,350.552872,2.946615,-0.038948,0.018000,0.980000,74.658381,...,101627.517578,2022-07-17 13:00:00,0.000000,415.419271,343.541821,-71.877450,-71.877450,-74.785118,47.709363,0.645768
2,01:30,wrf-ctsm,region,0.000000,345.692520,-6.751510,1.175791,0.300000,1.000000,0.000000,...,101489.017578,2022-07-17 13:00:00,0.000000,404.872779,345.692520,-59.180258,-59.180258,-53.604539,54.397218,0.699990
3,01:30,wrf-ctsm,sp,0.000000,347.329132,-5.905338,0.274503,0.300000,1.000000,0.000000,...,101632.197266,2022-07-17 13:00:00,0.000000,409.267839,347.329132,-61.938707,-61.938707,-56.307873,52.112415,0.787207
4,13:30,wrf,region,761.343674,369.882721,261.777489,34.837578,0.166591,0.971141,-133.966965,...,101181.816406,2022-07-18 01:00:00,126.832975,524.906627,359.208290,-165.698337,468.812361,172.197294,25.464890,3.750203


In [34]:
df_time_average[(df_time_average['location'] =='region')][['TSK', 'EMISS', 'LWUP', 'model', 'tag', 'location']]

,TSK,EMISS,LWUP,model,tag,location
0,292.913811,0.971141,405.344946,wrf,01:30,region
2,290.692535,1.000000,404.872779,wrf-ctsm,01:30,region
4,312.467010,0.971141,524.906627,wrf,13:30,region
6,312.350090,1.000000,539.696499,wrf-ctsm,13:30,region


# check with LILAC-CTSM data

In [6]:
ds_lilac_ctsm = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_wrf-ctsm/lnd/ctsm_lilac4.clm2.h0.2022-07-15-03600.nc')
ds_lilac_ctsm

<xarray.Dataset> Size: 54MB
Dimensions:       (levgrnd: 5, levsoi: 4, levlak: 10, levdcmp: 1, time: 120,
                   lon: 57, lat: 45, ltype: 9, numrad: 2)
Coordinates:
  * levgrnd       (levgrnd) float32 20B 0.05 0.25 0.7 1.5 2.5
  * levsoi        (levsoi) float32 16B 0.05 0.25 0.7 1.5
  * levlak        (levlak) float32 40B 0.05 0.6 2.1 4.6 ... 25.6 34.33 44.78
  * levdcmp       (levdcmp) float32 4B 1.0
  * time          (time) datetime64[ns] 960B 2022-07-15T00:59:59.780273438 .....
  * lon           (lon) float32 228B 357.2 357.2 357.3 ... 358.2 358.2 358.2
  * lat           (lat) float32 180B 53.27 53.29 53.3 ... 53.73 53.74 53.75
Dimensions without coordinates: ltype, numrad
Data variables: (12/45)
    mcdate        (time) int32 480B ...
    mcsec         (time) int32 480B ...
    mdcur         (time) int32 480B ...
    mscur         (time) int32 480B ...
    nstep         (time) int32 480B ...
    date_written  (time) |S16 2kB ...
    ...            ...
    WASTEHEAT     (time, lat, lon) float32 1MB ...
    WIND          (time, lat, lon) float32 1MB ...
    ZBOT          (time, lat, lon) float32 1MB ...
    PCT_LANDUNIT  (time, ltype, lat, lon) float32 11MB ...
    ALBGRD        (time, numrad, lat, lon) float32 2MB ...
    ALBGRI        (time, numrad, lat, lon) float32 2MB ...
Attributes: (12/102)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 07/02/25 13:51:46
    source:                               Community Terrestrial Systems Model
    hostname:                             
    ...                                   ...
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     hour_1
    Time_constant_3Dvars_filename:        ./ctsm_lilac4.clm2.h0.2022-04-01-03...
    Time_constant_3Dvars:                 ZSOI:DZSOI:WATSAT:SUCSAT:BSW:HKSAT:...

In [5]:
ds_lilac_ctsm['URBAN_AC'].max()

<xarray.DataArray 'URBAN_AC' ()> Size: 4B
array(0., dtype=float32)

In [26]:
var_list = ['FSDS', 'FSR', 'FLDS', 'FSH', 'FIRA', 'FIRE', 'URBAN_AC', 'WASTEHEAT', 'TSA', 'TSA_U', 'TSA_R', 'TG']
#var_list = ['QFX','T2', 'Q2', 'HGT', 'PBLH']
day_time = '13'
night_time = '01'
i1, i2 = 23, 36 # lon
j1, j2 = 12, 25 # lat
model_list = ['wrf', 'wrf-ctsm']
location_list = ['sp', 'region']
# Get the time index from ds_wrf (assumes both have the same time)
time_list = []
for date in ['07-16', '07-17', '07-18', '07-19']:
    for date_time in [day_time, night_time]:
        datetime = pd.to_datetime(f'2022-{date} {date_time}:00:00')
        time_idx = ds_lilac_ctsm.sel(time=datetime, method='nearest')
        time_idx2 = ds_lilac_ctsm.sel(time=datetime + pd.Timedelta(hours=1), method='nearest')
        rows_sp = []
        rows_region = []
        for var in var_list:
            mean_data_region = ((time_idx[var][i1:i2, j1:j2].mean().item()) + (time_idx2[var][i1:i2, j1:j2].mean().item())) / 2
            mean_data_sp = (time_idx[var][19, 30].item() + time_idx2[var][19, 30].item()) / 2
            rows_sp.append(mean_data_sp)
            rows_region.append(mean_data_region)
        df_rows = pd.DataFrame([rows_sp, rows_region], columns=var_list)
        #df_rows['time'] = datetime
        df_rows['location'] = location_list
        df_rows['tag'] = ['13:30', '13:30'] if date_time == day_time else ['01:30', '01:30']
        time_list.append(df_rows)
df_time = pd.concat(time_list, ignore_index=True)        
df_time_mean = df_time.groupby(['tag', 'location']).mean().reset_index()
df_time_mean

,tag,location,FSDS,FSR,FLDS,FSH,FIRA,FIRE,URBAN_AC,WASTEHEAT,TSA,TSA_U,TSA_R,TG
0,01:30,region,0.000000,0.000000,338.154556,-17.224276,56.007790,394.162342,0.0,0.004363,290.614014,291.826832,289.770092,290.081669
1,01:30,sp,0.000000,0.000000,347.467144,-5.498426,62.443589,409.910728,0.0,0.000000,292.913578,293.162464,291.045712,292.441383
2,13:30,region,776.482307,120.651557,365.657803,298.965298,166.479094,532.136890,0.0,0.000000,303.313702,303.604065,303.125019,311.817074
3,13:30,sp,757.155701,100.252165,371.723522,276.216063,173.557632,545.281158,0.0,0.000000,304.623497,304.679146,304.205830,313.943939


In [ ]:
df_time_mean['emissivity'] = df_time_mean['FIRE'] / (5.67e-8 * np.power(df_time_mean['TG'],4)) # this is not correct, should use TSKIN instead of TG
df_time_mean

,tag,location,FSDS,FSR,FLDS,FSH,FIRA,FIRE,URBAN_AC,WASTEHEAT,TSA,TSA_U,TSA_R,TG,emissivity
0,01:30,region,0.000000,0.000000,338.154556,-17.224276,56.007790,394.162342,0.0,0.004363,290.614014,291.826832,289.770092,290.081669,0.981773
1,01:30,sp,0.000000,0.000000,347.467144,-5.498426,62.443589,409.910728,0.0,0.000000,292.913578,293.162464,291.045712,292.441383,0.988441
2,13:30,region,776.482307,120.651557,365.657803,298.965298,166.479094,532.136890,0.0,0.000000,303.313702,303.604065,303.125019,311.817074,0.992752
3,13:30,sp,757.155701,100.252165,371.723522,276.216063,173.557632,545.281158,0.0,0.000000,304.623497,304.679146,304.205830,313.943939,0.989986


In [14]:
df_time_average = pd.read_csv('./data_for_figure/fluxs.csv')

In [24]:
check_wrf_ctsm = df_time_average[df_time_average['model']=='wrf-ctsm'][['tag', 'location', 'TSK', 'LWUP', 'EMISS', 'netLW', 'GLW', 'SWUP', 'SWDOWN', 'GRD']]
check_wrf_ctsm['LWup_modified'] = check_wrf_ctsm['LWUP'] * df_time_mean['emissivity'].values
check_wrf_ctsm

,tag,location,TSK,LWUP,EMISS,netLW,GLW,SWUP,SWDOWN,GRD,LWup_modified
2,01:30,region,290.692535,404.872779,1.0,-59.180258,345.692520,0.000000,0.000000,-53.604539,397.493021
3,01:30,sp,291.484810,409.304734,1.0,-61.837587,347.467148,0.000000,0.000000,-56.599385,404.573727
6,13:30,region,312.350090,539.696499,1.0,-168.965855,370.730644,107.089537,757.197693,115.952381,535.784743
7,13:30,sp,313.037651,544.464243,1.0,-172.740717,371.723526,100.670828,757.155670,115.927223,539.011936


In [21]:
df_time_average[df_time_average['model']=='wrf'][['tag', 'location', 'TSK', 'LWUP', 'EMISS', 'netLW', 'GLW', 'SWUP', 'SWDOWN']]

,tag,location,TSK,LWUP,EMISS,netLW,GLW,SWUP,SWDOWN
0,01:30,region,292.913811,405.344946,0.971141,-66.545977,348.866905,0.000000,0.000000
1,01:30,sp,294.085915,415.629189,0.980000,-71.501157,351.151047,0.000000,0.000000
4,13:30,region,312.467010,524.906627,0.971141,-165.698337,369.882721,126.832975,761.343674
5,13:30,sp,313.137489,534.255994,0.980000,-171.298760,370.364517,118.650236,766.611328
